# Japan Search Gallery client

> Tools to interact with the Japan Search Gallery

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pprint import pprint
import os
import json

class Component:
    
    def export(self):
        '''
        エクスポート

        Returns:
        * dict containing representation of the gallery
        '''
        return self.item

    def setMetadata(self, key, lang, value):
        '''
        メタデータの設定

        Parameters:
        * `key` - title, description, etc...
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'
        * `value` - 値
        '''
        g = self.g
        if key not in g:
            g[key] = {}
        g[key][lang] = value


    def save(self, path):
        '''
        JSONファイルの保存

        Parameters:
        * `path` - 出力先のパス
        '''
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            json.dump(self.g, f, ensure_ascii=False, indent=2)

class Media(Component):

    def __init__(self, type, title, source):
        self.item = {
            "imgType": type,
            "title": {
                "ja": title
            },
            "source": {
                "ja": source
            },
            "contentsRightsType": "incr",
        }

class IIIF(Media):
    def __init__(self, manifestUrl, infoJsonUrl, title, source):
        super().__init__("IIIF", title, source)

        item = self.item
        item["cover"] = None
        item["sourceUrl"] = None
        item["manifestUrl"] = manifestUrl
        item["infoJsonUrl"] = infoJsonUrl

class UrlImage(Media):

    def __init__(self, url, title, source, thumbnailUrl=None):
        super().__init__("URL", title, source)

        item = self.item

        item["url"] = url
        if thumbnailUrl is not None:
            item["thumbnailUrl"] = thumbnailUrl

class Part(Component):

    configType = {
        "text": "jps-curation-text",
        "list": "jps-curation-list",
    }

    def __init__(self, type, viewType=None):
        item = {}
        self.item = item
        item["cpid"] = "a"

        configType = self.configType

        type = configType[type] if type in configType else type
        item["type"] = type
        
        if type == "jps-curation-list":

            if viewType:
                item["viewType"] = viewType

            item["parts"] = []

    def insertText(self, lang, value):
        '''
        削除予定
        '''
        item = self.item

        if lang not in item:
            item[lang] = {
                "ops": []
            }

        ops = item[lang]["ops"]
        ops.append({
            "insert": value
        })
        ops.append({
            "insert": "\n"
        })

    def addOps(self, key, lang, value):
        '''
        opsに要素を追加します。

        Parameters:
        * `key` - title, description, etc...
        * `lang` - one of 言語, eg: 'ja', 'en', 'yomi'
        * `value` - 値
        '''
        item = self.item
        if key not in item:
            item[key] = {}

        if lang not in item[key]:
            item[key][lang] = {
                "ops": []
            }

        ops = item[key][lang]["ops"]
        ops.append({
            "insert": value
        })
        ops.append({
            "insert": "\n"
        })

    def setMetadata(self, key, lang, value):
        item = self.item
        if key not in item:
            item[key] = {}

        # self.item[key] = value
        item[key][lang] = value

    def setUrl(self, url):
        self.item["url"] = url

    def addPart(self, part):
        self.item["parts"].append(part.export())    

    def setImage(self, image):
        self.item["image"] = image.export()

class JPSGalleryClient(Component):

    g = {
        "version": 1,
        "seqNo": 0,
        "primaryTerm": 1,
        "curationType": "GALLERY",
        "title": {},
        "summary": {},
        "responsibility": {},
        "workspace": "abc",
        "workspaceTags": [],
        "permission": {
            "publicationLevel": "WORKSPACE",
            "level": "ADMIN"
        },
        "edit": {
            "userId": "system",
            "orgId": "jpsws",
            "date": 1663722096491,
            "action": "CREATE"
        },
        "parts": []
    }

    def __init__(self):
        self.g["id"] = "aaa"

    def setTitle(self, key, value):
        self.g["title"][key] = value

    def setSummary(self, key, value):
        self.g["summary"][key] = value

    

    def setTag(self, tag):
        g = self.g
        g["workspaceTags"].append(tag)

    def setImage(self, image):
        self.g["image"] = image.export()

First import the JPSGalleryClient class.
```
from jps_gallery_tools.api import JPSGalleryClient
```
To initialise the JPSGalleryClient you need to provide the base url of your Omeka instance's API.

In [ ]:
gallery = JPSGalleryClient()

## ギャラリーのメタデータの設定

ギャラリーのメタデータを設定します。

In [ ]:
show_doc(JPSGalleryClient.setMetadata)

---

### Component.setMetadata

>      Component.setMetadata (key, lang, value)

メタデータの設定

Parameters:
* `key` - title, description, etc...
* `lang` - one of 言語, eg: 'ja', 'en', 'yomi'
* `value` - 値

タイトルの設定

In [ ]:
gallery.setMetadata("title", "ja", "日本語タイトル")
gallery.setMetadata("title", "en", "English Title")
gallery.setMetadata("title", "yomi", "にほんごのよみ")

summaryの設定

In [ ]:
gallery.setMetadata("summary", "ja", "日本語の概要文")
gallery.setMetadata("summary", "en", "English Summary")

## Save json

設定した内容をjsonファイルに出力します。

In [ ]:
show_doc(JPSGalleryClient.save)

---

### JPSGalleryClient.save

>      JPSGalleryClient.save (path)

In [ ]:
gallery.save("tmp/gallery.json")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()